In [11]:
#RQ1: Step 1 - Setting up a prior belief based on targeted question

#targeted questions
question1 = "Do you prefer direct next steps or detailed discussion first?"
question1_options=["1. direct steps", "2. detailed discussion"]

question2= "Would you prefer a checklist or conversation?"
question2_options=["1. checklist", "2. conversation"]

In [12]:
#Function for asking question

def ask_question(question, answer):
    #question 1
    print(question)
    for i in range(len(answer)):
        print(answer[i])
    choice= input('Enter your preferred answer number') #change it to better UI based button later
    return answer[int(choice)-1]


choice_result = []

choice_result.append(ask_question(question1, question1_options))
choice_result.append(ask_question(question2, question2_options))

print(choice_result)


Do you prefer direct next steps or detailed discussion first?
1. direct steps
2. detailed discussion
Would you prefer a checklist or conversation?
1. checklist
2. conversation
['1. direct steps', '1. checklist']


In [ ]:
#creating user profile based on user's answer

user_profile = {"preferred_conversation_style": None}

#take 2 answers and based on that assign key-value pair for user profile--think


if choice_result[0]=="1. direct steps" and choice_result[1] == "1. checklist":
    user_profile["preferred_conversation_style"] = "action_based"
elif choice_result[0]=="2. detailed discussion" and choice_result[1] == "2. conversation":
    user_profile["preferred_conversation_style"] = "relationship_based"
else:
    user_profile["preferred_conversation_style"] = "mixed"
        
print(user_profile)


{'preferred_conversation_style': 'mixed'}
